In [21]:
import pandas as pd

In [35]:
import requests
import pandas as pd
import time

# Configurações do Mapbox
ACCESS_TOKEN = 'pk.eyJ1IjoicXVlaXJvenZhbGUiLCJhIjoiY2pwZGhoMGNhMDNvbDNyazIwM3ViMzI0biJ9.fFw10jxOAWBYUZA_Nbd1JA'  # Substitua pelo seu token de acesso
STYLE = 'mapbox/streets-v11'  # Escolha o estilo do mapa desejado

file_path = '/mnt/d/radio/gcm/DSDPlus.LRRP'
TOKEN = "661891149:AAHXbPBwMSR5ZIeySyPs-y4L4PvRvzJvozU"
CHAT_ID = "-1002022196761"


def get_color_by_id(id):
    colors = {
        1043: 'e41621', 1036: '49bb96', 1025: 'e70cff', 1033: 'f9c45e',
        1042: '671837', 1047: 'b8a23e', 1037: '5fcd3d', 1029: '4b4851',
        1030: '37c7b0', 1038: 'ed5fa2', 1007: '758527', 1049: '5dbe94',
        2003: 'c909a6', 1039: '8caf73', 1027: '80aa3c', 1022: 'df89ba',
        1009: 'bfa07a', 1012: 'e29eb3', 1018: 'c06916', 1032: '1692e3',
        1024: 'c4243f', 1019: '8e41ad', 1031: 'c5f9a7', 1016: '05be41',
        1010: '72ed96', 1028: '23cd20', 1023: '65c199', 1003: '255dfe',
        1021: '199b50', 1026: '1675c9', 1014: '1f95d9', 1005: '64deb1',
        1051: 'ccfc84'
    }
    return colors.get(id, "00FF00")  # Cor padrão: verde

def process_data(file_path):
    data = pd.read_csv(file_path, delimiter='\s+', header=None, names=["Data", "Hora", "id", "lat", "long", "velocidade", "direção"])
    data.sort_values(by=['id', 'Data', 'Hora'], inplace=True)
    return data.tail(10)

def fetch_map_image(data, id, size="800x800"):
    coordinates = ';'.join([f"{row['long']},{row['lat']}" for index, row in data.iterrows()])
    color = get_color_by_id(id)
    path = f"path-5+{color}({coordinates})"
    center_lon = data['long'].mean()
    center_lat = data['lat'].mean()
    zoom = 12
    url = f"https://api.mapbox.com/styles/v1/{STYLE}/static/{path}/{center_lon},{center_lat},{zoom}/{size}?access_token={ACCESS_TOKEN}"
    print(url)
    response = requests.get(url)
    if response.status_code == 200:
        map_filename = f'map_id_{id}.png'
        with open(map_filename, 'wb') as f:
            f.write(response.content)
        return map_filename
    else:
        print(f"Erro ao buscar imagem do mapa para ID {id}:", response.content)
        return None

def send_map(token, chat_id, map_file, id):
    url = f"https://api.telegram.org/bot{token}/sendPhoto"
    with open(map_file, "rb") as image_file:
        files = {'photo': image_file}
        data = {'chat_id': chat_id, 'caption': f'Mapa da rota para ID {id}', 'parse_mode': 'Markdown'}
        response = requests.post(url, data=data, files=files)
        print(f"Resposta do servidor ao enviar o mapa do ID {id}:", response.json())

# Execução
if __name__ == "__main__":
    data = process_data(file_path)
    grouped_data = data.groupby('id')
    for id, group in grouped_data:
        if group.shape[0] < 2:
            continue  # Ignora IDs com menos de dois pontos
        map_file = fetch_map_image(group, id)
        if map_file:
            send_map(TOKEN, CHAT_ID, map_file, id)

https://api.mapbox.com/styles/v1/mapbox/streets-v11/static/path-5+c909a6(-47.29197,-23.27707;-47.29802,-23.28261;-47.29766,-23.2882;-47.28988,-23.28449;-47.28751,-23.27631;-47.28754,-23.27634;-47.28758,-23.27639;-47.28801,-23.27609;-47.2842,-23.28322;-47.28762,-23.26198)/-47.289999,-23.27827,12/800x800?access_token=pk.eyJ1IjoicXVlaXJvenZhbGUiLCJhIjoiY2pwZGhoMGNhMDNvbDNyazIwM3ViMzI0biJ9.fFw10jxOAWBYUZA_Nbd1JA
Resposta do servidor ao enviar o mapa do ID 2003: {'ok': True, 'result': {'message_id': 3232, 'from': {'id': 661891149, 'is_bot': True, 'first_name': 'X9', 'username': 'h4Lu1zQu3iroz_bot'}, 'chat': {'id': -1002022196761, 'title': 'MaeRussia', 'type': 'supergroup'}, 'date': 1714911200, 'photo': [{'file_id': 'AgACAgEAAx0EeIhGGQACDKBmN3fgB9_MvqPXOnXT384rfpMIUAACcKwxG6RHuUU8Hi6Z3VMvoQEAAwIAA3MAAzUE', 'file_unique_id': 'AQADcKwxG6RHuUV4', 'file_size': 967, 'width': 90, 'height': 90}, {'file_id': 'AgACAgEAAx0EeIhGGQACDKBmN3fgB9_MvqPXOnXT384rfpMIUAACcKwxG6RHuUU8Hi6Z3VMvoQEAAwIAA20AAzUE', 

In [29]:
data = process_data(file_path)
if not data.empty:
    map_file = fetch_map_image(data)
    if map_file:
        send_map(TOKEN, CHAT_ID, map_file)
    else:
        print("Erro ao gerar o mapa.")
else:
    print("Nenhum dado novo para processar.")

URL do mapa: https://api.mapbox.com/styles/v1/mapbox/streets-v11/static/path-5+c909a6(-47.29071,-23.27157;-47.29589,-23.26637;-47.3071,-23.25616;-47.30705,-23.2561;-47.29576,-23.27065;-47.28914,-23.2802;-47.28913,-23.28032;-47.29193,-23.27714;-47.29191,-23.27719;-47.29195,-23.27716;-47.29197,-23.27707;-47.29802,-23.28261;-47.29766,-23.2882;-47.28988,-23.28449;-47.28751,-23.27631;-47.28754,-23.27634;-47.28758,-23.27639;-47.28801,-23.27609;-47.2842,-23.28322;-47.28762,-23.26198)/-47.292528,-23.274777999999998,12/800x800?access_token=pk.eyJ1IjoicXVlaXJvenZhbGUiLCJhIjoiY2pwZGhoMGNhMDNvbDNyazIwM3ViMzI0biJ9.fFw10jxOAWBYUZA_Nbd1JA
Resposta do servidor ao enviar o mapa: {'ok': True, 'result': {'message_id': 3221, 'from': {'id': 661891149, 'is_bot': True, 'first_name': 'X9', 'username': 'h4Lu1zQu3iroz_bot'}, 'chat': {'id': -1002022196761, 'title': 'MaeRussia', 'type': 'supergroup'}, 'date': 1714910487, 'photo': [{'file_id': 'AgACAgEAAx0EeIhGGQACDJVmN3UX23plqfadRHLuLtOmovlt_gACb6wxG6RHuUWEvQhvTc

In [21]:
data
-47.28754,-23.27634	
-47.28758,-23.27639	
-47.28801,-23.27609

,Data,Hora,id,lat,long,velocidade,direção
4792,2024/05/04,9:18:14,2003,-23.27634,-47.28754,0.028,52.0
4806,2024/05/04,9:19:53,2003,-23.27639,-47.28758,0.028,52.0
4813,2024/05/04,9:21:55,2003,-23.27609,-47.28801,23.484,136.0
4862,2024/05/04,9:34:07,2003,-23.28322,-47.28420,0.028,286.0
4944,2024/05/04,9:58:00,2003,-23.26198,-47.28762,47.222,262.0


In [24]:
import pandas as pd
import requests

# Função para ler os dados
def read_data(file_path):
    data = pd.read_csv(file_path, names=["Datetime", "ID", "Lat", "Long", "Speed"], skiprows=1)
    data['Datetime'] = pd.to_datetime(data['Datetime'])
    data.sort_values(by='Datetime', inplace=True)
    return data

# Função para gerar a imagem do mapa usando a API do Mapbox
def fetch_map_image(data, style, access_token, size="800x800"):
    grouped = data.groupby('ID')
    paths = []

    for id, group in grouped:
        if len(group) < 2:
            continue
        start_point = group.iloc[0]
        end_point = group.iloc[-1]
        path = f"path-5+ff0000({start_point['Long']},{start_point['Lat']},{end_point['Long']},{end_point['Lat']})"
        paths.append(path)

    center_lon = data['Long'].mean()
    center_lat = data['Lat'].mean()
    zoom = 12

    all_paths = ','.join(paths)
    url = f"https://api.mapbox.com/styles/v1/{style}/static/{all_paths}/{center_lon},{center_lat},{zoom}/{size}?access_token={access_token}"
    
    response = requests.get(url)
    if response.status_code == 200:
        with open('map.png', 'wb') as f:
            f.write(response.content)
        return 'map.png'
    else:
        return "Failed to fetch map image"

# Configurações
STYLE = "your-mapbox-style-here"
ACCESS_TOKEN = "your-mapbox-access-token-here"
FILE_PATH = "your-data-file-path-here.csv"

# Processamento principal
data = read_data(FILE_PATH)
map_image_file = fetch_map_image(data, STYLE, ACCESS_TOKEN)
print("Map image is saved as:", map_image_file)


33


In [25]:
data = {}
for id in unic_ids:
    data[id]=generate_random_hex_color()

In [26]:
data

{1043: 'e41621',
 1036: '49bb96',
 1025: 'e70cff',
 1033: 'f9c45e',
 1042: '671837',
 1047: 'b8a23e',
 1037: '5fcd3d',
 1029: '4b4851',
 1030: '37c7b0',
 1038: 'ed5fa2',
 1007: '758527',
 1049: '5dbe94',
 2003: 'c909a6',
 1039: '8caf73',
 1027: '80aa3c',
 1022: 'df89ba',
 1009: 'bfa07a',
 1012: 'e29eb3',
 1018: 'c06916',
 1032: '1692e3',
 1024: 'c4243f',
 1019: '8e41ad',
 1031: 'c5f9a7',
 1016: '05be41',
 1010: '72ed96',
 1028: '23cd20',
 1023: '65c199',
 1003: '255dfe',
 1021: '199b50',
 1026: '1675c9',
 1014: '1f95d9',
 1005: '64deb1',
 1051: 'ccfc84'}

In [31]:
get_color_by_id(10326)

10326


'00FF00'